In [0]:
# importing the libraries
import pandas as pd
import numpy as np
import h5py
import os
from google.colab import drive
import tensorflow as tf
from sklearn.model_selection import train_test_split

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, Flatten
from torch.optim import Adam, SGD

In [0]:
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls
os.chdir('gdrive/My Drive/My_AI_proj/CERN_data/')
!pwd

gdrive	sample_data
/content/gdrive/My Drive/My_AI_proj/CERN_data


In [0]:

photon='SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
electron='SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'


In [0]:
class Network(Module):   
  def __init__(self):
    super(Network, self).__init__()
    self.conv1=Conv2d(2,32,3)
    self.conv2=Conv2d(32,32,3)

    self.fc1=Linear(in_features=32*13*13, out_features=1500)
    self.fc2=Linear(in_features=1500, out_features=500)
    self.do1=Dropout(p=0.4)
    self.fc3=Linear(in_features=500, out_features=100)
    self.do2=Dropout(p=0.3)
    self.out=Linear(in_features=100, out_features=1)

  # Defining the forward pass    
  def forward(self, x):
    x=self.conv1(x)
    x=ReLU(x)
    x=self.conv2(x)
    x=ReLU(x)
    x=MaxPool2d(x,kernel_size=2,stride=1)
    x=x.reshape(-1,64,13,13)
    x=self.fc1(x)
    x=ReLU(x)
    x=self.fc2(x)
    x=ReLU(x)
    x=self.do1(x)
    x=self.fc3(x)
    x=ReLU(x)
    x=self.do2(x)
    x=self.out(x)
    return x

In [0]:
network = Network()
# defining the optimizer
optimizer = Adam(network.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    network = network.cuda()
    criterion = criterion.cuda()
    
print(network.conv1.weight.shape)

torch.Size([32, 2, 3, 3])


In [0]:
def input_target(photon, electron):
  X_input=[]
  electron_image_data=h5py.File(electron,'r')['X'].value
  for i in tqdm(range(len(electron_image_data))):
    image=electron_image_data[i]
    X_input.append(image)
  del(electron_image_data)
  photon_image_data=h5py.File(photon, 'r')['X'].value
  for i in tqdm(range(len(photon_image_data))):
    image=photon_image_data[i]
    X_input.append(image)
  y_photon=[1.0]*249000
  y_electron=[0.0]*249000
  y_target=np.array(y_photon+y_electron)
  X_input=np.asarray(X_input)
  print(X_input.shape,y_target.shape)
  X_train, X_test, y_train, y_test=train_test_split(X_input,y_target, test_size=0.2,random_state=23)
  return X_train, X_test, y_train, y_test


In [0]:
X_train, X_test, y_train, y_test=input_target(photon,electron)

100%|██████████| 249000/249000 [00:00<00:00, 1792785.30it/s]


(498000, 32, 32, 2) (498000,)


In [0]:
print(X_train.shape, y_train.shape, (X_train.size*X_train.itemsize)/1048576)

(323700, 32, 32, 2) (323700,) 2528.90625


In [0]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [0]:
def tensor_data_generator(X, y, validation=0):
  

In [0]:
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

In [0]:
train_loader=torch.utils.data.DataLoader([X_train,y_train], batch_size=100)

In [0]:

data,label=next(iter(train_loader))

RuntimeError: ignored

In [0]:
def train(epoch):
    network.train()
    tr_loss = 0
    dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
    # getting the training set
    x_train, y_train = Variable(X_train.astype(dtype)), Variable(y_train.astype(dtype))
    # getting the validation set
    x_val, y_val = Variable(X_test), Variable(y_test)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the network parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = network(x_train)
    output_val = network(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)
        # defining the number of epochs
n_epochs = 25
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

TypeError: ignored

In [0]:
del(train_loader)

NameError: ignored